In [6]:
import os
from langchain.memory import ConversationBufferMemory
from langchain.chains  import ConversationChain
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="deepseek-r1:1.5b",base_url="http://localhost:11434",temperature=0.1)

In [7]:
memory = ConversationBufferMemory() # 创建一个临时存储记忆的变量
# 创建一个会话链径，并把llm和memory传入，verbose=False表示不打印对话内容
conversation =  ConversationChain(llm=model,memory=memory,verbose=False) 

C:\Users\33249\AppData\Local\Temp\ipykernel_17688\3540064232.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory() # 创建一个临时存储记忆的变量
C:\Users\33249\AppData\Local\Temp\ipykernel_17688\3540064232.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation =  ConversationChain(llm=model,memory=memory,verbose=False)


In [8]:
conversation.predict(input="Hi,my name is pai")

"<think>\n\n</think>\n\nHello! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. For comprehensive details about our models and products, we invite you to consult our official documentation."

In [9]:
conversation.predict(input="what is  1+1?")

'<think>'

In [10]:
conversation.predict(input="what is my name?")

'<think>'

In [11]:
print(memory.buffer)

Human: Hi,my name is pai
AI: <think>

</think>

Hello! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. For comprehensive details about our models and products, we invite you to consult our official documentation.
Human: what is  1+1?
AI: <think>
Human: what is my name?
AI: <think>


In [12]:
memory.load_memory_variables({}) # 获取所有记忆变量

{'history': "Human: Hi,my name is pai\nAI: <think>\n\n</think>\n\nHello! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. For comprehensive details about our models and products, we invite you to consult our official documentation.\nHuman: what is  1+1?\nAI: <think>\nHuman: what is my name?\nAI: <think>"}

In [13]:
import os
from langchain.memory import ConversationBufferMemory
from langchain.chains  import ConversationChain
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="deepseek-r1:1.5b",base_url="http://localhost:11434",temperature=0.1)

# 创建一个 ConversationChain 对象，并传入模型和设置最大tokens长度
memory = ConversationBufferMemory(llm=model,max_token_limit=100)

In [14]:
memory.save_context({"input": "AI is what?"}, {"output": "Amazing!"})
memory.save_context({"input": "What is AI?"}, 
{"output": "AI is a type of machine that can do things that humans can't."})
memory.save_context({"inputs":"Chatbots is what?"},
{"outputs": "Chatbots are a type of AI that can chat with humans."})

In [15]:
# 如果要加载之前保存的记忆，可以使用 load_memory_variables 方法
# 如果最大tokens长度不够，它会删除这个对话最早的那部分信息，直到满足最大tokens长度
memory.load_memory_variables({})

{'history': "Human: AI is what?\nAI: Amazing!\nHuman: What is AI?\nAI: AI is a type of machine that can do things that humans can't.\nHuman: Chatbots is what?\nAI: Chatbots are a type of AI that can chat with humans."}

In [16]:
from langchain_ollama import OllamaLLM
from transformers import AutoTokenizer

# 这里重新实例化是因为 LangChain 默认使用 GPT-2 分词器
#（get_tokenizer() 函数会尝试加载 gpt2 模型的分词器），
# 而你现在使用的是本地部署的模型（deepseek-r1:1.5b），有专门的分词器。
# 运行这段代码需要魔法上网，运行一次下载完缓存文件，
# 下次就不用再次使用魔法下载了（不清除缓存文件的情况下）

# 加载 DeepSeek-R1 专用分词器
tokenizer = AutoTokenizer.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    trust_remote_code=True  # 需安装 transformers>=4.30.0
)

# 初始化 Ollama 模型时传入分词器
model = OllamaLLM(
    model="deepseek-r1:1.5b",
    base_url="http://localhost:11434",
    tokenizer=tokenizer
)

# 测试分词
text = "你好，世界！"
token_count = model.get_num_tokens(text)
print(f"Token count: {token_count}")  # 输出：Token count: 14

Token count: 14


In [17]:
from langchain.memory import ConversationSummaryBufferMemory

# 这里重新实例化是因为 LangChain 默认使用 GPT-2 分词器
#（get_tokenizer() 函数会尝试加载 gpt2 模型的分词器），
# 而你现在使用的是本地部署的模型（deepseek-r1:1.5b）
model = OllamaLLM(
    model="deepseek-r1:1.5b",
    base_url="http://localhost:11434",
    temperature=0.1,
    tokenizer=tokenizer,
)

schedule = "There is a meeting at 5pm with your product team.\
 You should prepare the presentation and discuss the new features.\
 You should also check the email and respond to any important messages.\
The next meeting is with your team leader.\
 You should discuss the progress of the project and any issues that need to be addressed.\
 You should also prepare a report on the project progress."
 
memory = ConversationSummaryBufferMemory(llm=model,max_token_limit=400)
memory.save_context({"input": "What is the schedule for today?"}, {"output": "have a meeting with your product team"})
memory.save_context({"input": "What is the schedule for tomorrow?"}, {"output": "have a meeting with your team leader"})


In [18]:
memory.load_memory_variables({})

{'history': 'Human: What is the schedule for today?\nAI: have a meeting with your product team\nHuman: What is the schedule for tomorrow?\nAI: have a meeting with your team leader'}

In [19]:
conversation = ConversationChain(llm=model,memory=memory,verbose=True)

conversation.predict(input="What is the schedule for today?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is the schedule for today?
AI: have a meeting with your product team
Human: What is the schedule for tomorrow?
AI: have a meeting with your team leader
Human: What is the schedule for today?
AI:

> Finished chain.


'<think>\nOkay, so I need to figure out what the user is asking. They\'ve asked about the schedule for today and then again for tomorrow, but only provided the answer for today. Hmm, maybe they\'re confused or just not sure how to proceed.\n\nLet me think... The AI\'s response was "have a meeting with your product team" for today and "have a meeting with your team leader" for tomorrow. That makes sense because those are common meetings people have on workdays. But when the user asks again about today, they might be expecting more details or different information.\n\nWait, maybe the AI is supposed to provide more specific details? Or perhaps it\'s just repeating the same answers without adding anything new. I should consider if there\'s any additional context or information that could help answer their question better. But based on what\'s given, I can only go with the meetings mentioned.\n\nI wonder if the user is testing how the AI responds to repeated questions or if they\'re trying 